# Image-Classification-For-Cross-Analysis-of-Chest-X-Ray

In [1]:
"""
Loads a pretrained CNN model and presents a Gradio app to the user.
The user can upload a chest X-Ray image which will be preprocessed similarly
to the trained model, then run through the model.
A prediction of 1 or more of 14 diagnoses or no diagnosis will be output.
This will be followed up with suggested treatment from an OpenAI API.
"""

# Standard imports
import pandas as pd
from pathlib import Path
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
from pathlib import Path
import gradio as gr
from gtts import gTTS
import random

# Gradio App Ensambling Aleksandar's and Patrick's Models

In [3]:
# first lets load our pre-trained models
aleksandar_model = tf.keras.models.load_model(Path("Resources/detection_model.keras"))
patrick_model =  tf.keras.models.load_model(Path("Resources/final_model.keras"))

d:\Development\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
# Load the CSV file that has list of all images, for random selection
df = pd.read_csv("Resources/Data_Entry_2017_v2020.csv")

In [23]:
# Define the load random function to load any of the images within the dataset (CSV)
def load_random(submit):
    idx = random.randrange(0, len(df["Image Index"]))
    img_name = df["Image Index"].iloc[idx]
    img_name = f"./Images/{img_name}"
    print(img_name)
    return img_name


In [24]:
# Make our gradio app compenents dynamic
def enable_submit(submit):
    return gr.update(interactive=True)

def disable_submit(submit):
    return gr.update(interactive=False)


In [32]:
# Define the function of our model that will process the image and meta data to make a prediction
def do_predict(img):
    # Process image based on preprocess code and prep to enter into model
    img = Image.fromarray(img)
    # setup model 1 image
    mod1img = img.resize((256, 256))
    mod1img = mod1img.convert('RGB')  # Ensure all images are in RGB format
    mod1img = np.array(mod1img)
    mod1img = mod1img / 255.0 # Normalize to range [0, 1]
    mod1img = np.array(mod1img)
    mod1img = np.expand_dims(mod1img, axis=0)
    # setup model 2 image
    mod2img = img.resize((224, 224))
    mod2img = mod2img.convert('RGB')  # Ensure all images are in RGB format
    mod2img = np.array(mod2img)
    mod2img = mod2img / 255.0 # Normalize to range [0, 1]
    mod2img = np.array(mod2img)
    mod2img = np.expand_dims(mod2img, axis=0)

    # make predictions with both models
    prediction1 = aleksandar_model.predict(mod1img)
    prediction2 = patrick_model.predict(mod2img)
    
    # take the average of the predictions
    prediction = (prediction1[0][0] + prediction2[0][0]) / 2
    
    # With a poor performing model we wanted a little better breakdown than a simple binary output
    # Using two break points on the sigmoid output to add in the ambiguity.
    if prediction < 0.50:
        output = f"This model found nothing in the X-ray. Disclaimer: This app should only be used by doctors as a cross reference with standard radiology."
    else:
        output = f"This model suspects a diagnosis in this X-ray. Human analysis is required to determine the exact diagnosis."

    # Create a wave file for the Text-to-speach output (hope simulateous connectsion do not cause problems here)
    wavobj = gTTS(text=output, lang="en", slow=False)
    wavobj.save("output.wav")
 
    return output, "output.wav"

In [ ]:
# Create an instance of the Gradio Blocks as a dynamic application

with gr.Blocks(theme="abidlabs/dracula_revamped") as app:
        title = gr.Markdown("# Demo&mdash;Chest X-Ray Cross Analysis")
        rand = gr.Button(value="Load Random Image from Dataset.")
        image = gr.Image(label="Or Upload Your Own.", value="PIL.Image.Image", interactive=True)
        submit = gr.Button(value="Classify Image", interactive=False)
        out_txt = gr.Textbox(lines=3, label="Text Prediction", show_copy_button=True)
        out_aud = gr.Audio(autoplay=False, label="Audio Prediction", interactive=False)
        image.change(enable_submit, inputs=submit, outputs=submit)
        image.clear(disable_submit, inputs=submit, outputs=submit)
        rand.click(load_random, inputs=rand, outputs=image)
        submit.click(do_predict, inputs=image, outputs=[out_txt, out_aud])
# Launch the app
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7868
Running on public URL: https://d1a5467b317cc940f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
./Images/00012012_000.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 600ms/step
./Images/00018253_060.png
./Images/00003523_001.png
./Images/00005460_003.png
./Images/00000512_002.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
./Images/00019928_000.png
./Images/00010620_002.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
./Images/00016421_001.png
./Images/00013111_024.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
./Images/00007502_000.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 504ms/step
./Images/00014900_000.png
./Images/00014196_003.png
./Images/00010433_008.png
./Images/00004808_080.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
1/1 ━━━━━━━━━━━━━━━━━